## Deep Equal Risk Pricing of Financial Derivatives with Multiple Hedging Instruments 
This notebook presents an example of implementation of the equal risk pricing framework as presented in Carbonneau and Godin (2021). 
- The equal risk option price $C_{0}^{\star}$ and the measure of market incompleteness $\epsilon^{\star}$ are computed for an at-the-money European-type put option of maturity 1 year under the Merton jump-diffusion model. 
- The deep hedging algorithm of Buehler et al. (2019) is applied twice to train two distinct long-short term memory which are used to approximate the optimal hedging strategy respectively for the long and the short position in the derivative.
- The convex risk measure used to quantity the residual risk is the Conditional Value-at-Risk with alpha = 0.95. 
- For a complete description of the algorithm, the reader is referred to Section 3 and to the pseudo-algorithm in Appendix B. 

Note: some parts of the code are inspired by the following implementation of the deep hedging algorithm: 
    - https://nbviewer.jupyter.org/urls/people.math.ethz.ch/~jteichma/lecture_ml_web/lecture_3.ipynb
    - https://github.com/mgroncki/DataScienceNotebooks/blob/master/DeepHedging/DeepHedging_Part1.ipynb  

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import datetime as dt
import numpy as np
np.seterr(divide = 'ignore') 
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam
from keras import backend as K
from scipy import stats
from scipy.stats import skew
from scipy.stats import norm

Using TensorFlow backend.


### 1) Deep hedging class
- This class implements the deep hedging algorithm with a long-short term memory (LSTM) under the CVaR risk measure
- Can be applied for hedging with the underlying (daily and month basis) or with two options.
- Works for both long and short position

In [3]:
class DeepAgent(object):
    """
    Inputs:
    nbs_point_traj       : if [S_0,...,S_N], nbs_point_traj = N+1
    batch_size           : Size of the batch
    nbs_input            : Nbs of features of the neural networks
        - This is prior to adding V_{t} at each-timestep
        - Depends on which asset (stock or options) is used
    loss_type            : Loss function applied on the rewards {MSE, SMSE}
    nbs_assets           : Number of hedging instruments
    position_type        : {Long, Short}
    hidden_layers        : Number of LSTM cells
    nbs_units            : Fixed number of units per layer
    lr                   : Learning rate
    prepro_stock         : preprocessing of stock prices {Log-moneyness, Nothing}.
    hedging_instruments  : {"Stock", "ATM call and put"}
    name                 : Model name to be saved
    alpha                : risk aversion parameter of CVaR
    """
    def __init__(self, nbs_point_traj, batch_size, nbs_input, loss_type, nbs_assets, position_type,
                 nbs_layers, nbs_units, lr, prepro_risky_assets, hedging_instruments, alpha, name='model'):
        
        tf.reset_default_graph()
        self.nbs_point_traj            = nbs_point_traj
        self.batch_size                = batch_size
        self.nbs_input                 = nbs_input
        self.loss_type                 = loss_type
        self.nbs_assets                = nbs_assets
        self.position_type             = position_type
        self.nbs_layers                = nbs_layers
        self.nbs_units                 = nbs_units
        self.lr                        = lr
        self.prepro_stock              = prepro_stock
        self.hedging_instruments       = hedging_instruments
        self.alpha                     = alpha
        self.strike                    = tf.ones(self.batch_size)*strike
        
        # 1) Placeholder - Stock price is normalized!
        # self.input:
        # - [S_t, IV_t, ..., T-t];
        #   - S_t  : price of the underlying 
        #   - IV_t : implied vol
        self.input        = tf.placeholder(tf.float32, [nbs_point_traj, batch_size, nbs_input])  # Stock prices are normalized here                   
        self.disc_tensor  = tf.placeholder(tf.float32, [nbs_point_traj, batch_size, 1]) # To discount from 't' to 0.
        self.deltas       = tf.zeros(shape = [nbs_point_traj-1, batch_size, 1], dtype=tf.float32)
                
        if(self.hedging_instruments == "ATM call and put"):
            self.opts_price = tf.placeholder(tf.float32, [nbs_point_traj, batch_size, 2])
        self.strategy  = tf.zeros(shape = [nbs_point_traj-1, batch_size, nbs_assets], dtype=tf.float32)
    
        # 2) Store prices of the underlying unormalized
        self.underlying_unorm_prices = self.inverse_processing(self.input[:,:,0])   
         
        # 3) Store the unormalized hedging instrument prices
        if(self.nbs_assets ==1):
            if(self.hedging_instruments == "Stock"):
                self.unorm_hedging_inst_price_b = tf.expand_dims(self.inverse_processing(self.input[0:-1,:,0]),axis=2)

                # Here, end of price is the same as the beginning of the next period
                self.unorm_hedging_inst_price_e = tf.expand_dims(self.inverse_processing(self.input[1:,:,0]),axis=2)

        elif(self.nbs_assets > 1):
            if(self.hedging_instruments == "ATM call and put"):
                self.unorm_hedging_inst_price_b = self.opts_price[0:-1,:]  # all prices except last one, useless
                self.unorm_hedging_inst_price_e = self.payoff_liquid_inst_func()
                  
        # 4) Discounted difference prices of the hedging instruments
        unorm_discount_hedging_inst_price_b = self.unorm_hedging_inst_price_b*self.disc_tensor[0:-1,:,:]
        unorm_discount_hedging_inst_price_e = self.unorm_hedging_inst_price_e*self.disc_tensor[1:,:,:]
        dS_hedging_inst = unorm_discount_hedging_inst_price_e - unorm_discount_hedging_inst_price_b 
        
        # 5) Compute the hedging strategy for each time-step with the LSTM
        lstm = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers = self.nbs_layers, num_units  = self.nbs_units)
         
        # 5.1) Output layer - to produce the number of stocks to hold
        W_o = tf.get_variable(name='W_o', shape=[self.nbs_assets, self.nbs_units], initializer=tf.contrib.layers.xavier_initializer())
        b_o = tf.get_variable(name='b_o', shape=[self.nbs_assets], initializer=tf.constant_initializer(0.))
        activation_out = None
            
        # 5.2) Portfolio value - updated at each time-step:
        # - Initial price is zero
        V_t = tf.zeros(self.batch_size)
            
        # 5.3) Initial time-step - must be different to include the initial-state
        for t in range(self.nbs_point_traj-1): 
            input_t = tf.expand_dims(tf.concat([self.input[t,:,:], tf.expand_dims(V_t, axis = 1)], axis=1), axis = 0)
            if(t==0):
                h1, final_state = lstm(input_t)
                    
                # With an LSTM, the output is of shape [1, BS, 1], no need to expand_dims self.strategy
                self.strategy = tf.tensordot(h1, W_o, axes=[[2], [1]]) + b_o
                if activation_out is not None:
                    self.strategy = activation_out(self.strategy)
                        
            else:
                h1, final_state = lstm(input_t, initial_state = final_state)
                output = tf.tensordot(h1, W_o, axes=[[2], [1]]) + b_o
                if activation_out is not None:
                    output = activation_out(output)
                
                # Store the resulting strategy in self.strategy
                self.strategy = tf.concat([self.strategy, output], axis = 0)
                    
                # Compute the value for the next time-period
                factor = tf.div(self.disc_tensor[t,:,0],self.disc_tensor[t+1,:,0]) # equal exp(rh)                 
                # update rule: see Remark 2.1
                V_t_pre = V_t
                V_t = V_t_pre*factor + tf.reduce_sum(self.strategy[t,:,:]*(self.unorm_hedging_inst_price_e[t,:,:] - self.unorm_hedging_inst_price_b[t,:,:]*tf.expand_dims(factor, axis=1)), axis=1)
          
        # 6) Compute the payoff of the put to hedge 
        self.payoff = tf.maximum(self.strike - self.underlying_unorm_prices[-1,:],0)   
        
        # 7) Compute hedging errors
        cumulative_factor = tf.reciprocal(self.disc_tensor[-1,:,0])  # Project from 0 to T
        self.disc_hedging_gain = tf.reduce_sum(dS_hedging_inst*self.strategy, axis=[0,2]) # should be of [batch_size]
        
        if(self.position_type == 'Short'):
            self.hedging_err = self.payoff - cumulative_factor*self.disc_hedging_gain
        elif(self.position_type == 'Long'):
            self.hedging_err = - self.payoff - cumulative_factor*(self.disc_hedging_gain)
        
        # 8) Compute the CVaR_{alpha} on the batch of hedging error
        # - This is the empirical cost functions estimated with a mini-batch
        # - Equivalent to estimator of CVaR_{alpha} presented in our paper
        self.loss = tf.reduce_mean(tf.contrib.framework.sort(self.hedging_err)[tf.cast(self.alpha*self.batch_size,tf.int32):])
        
        # 9) SGD step with the adam optimizer
        optimizer  = tf.train.AdamOptimizer(learning_rate = lr)  
        self.train = optimizer.minimize(self.loss) 
       
        # 10) Save the model
        self.saver = tf.train.Saver()
        self.model_name = name
        
    # Given a type of preprocessing, reverse the processing of the stock price
    def inverse_processing(self, paths):
        if (self.prepro_stock == "Log-moneyness"):
            paths = self.strike*tf.exp(paths)
        return paths
        
    # ---------------------------------------------------------------------------------------# 
    # Function to call the deep hedging algorithm batch-wise
    # Function adapted from https://github.com/mgroncki/DataScienceNotebooks/blob/master/DeepHedging/DeepHedging_Part1.ipynb 
    """
    Input:
     - paths       : tensor of features of dimension [n_timesteps+1, n_sims, nbs_features]
     - disc_batch  : tensor of discount factors of dimension [n_timesteps+1, N_batch, 1]
     - epochs      : total number of epochs to run
    """
    def train_deephedging(self, paths, disc_batch, sess, epochs, option_prices=-99999999999):
        sample_size       = paths.shape[1]               # total number of paths in the train set
        batch_size        = self.batch_size    
        idx               = np.arange(sample_size)       # [0,1,...,sample_size-1]
        start             = dt.datetime.now()            # Time-to-train
        self.loss_epochs  = 9999999*np.ones(epochs)      # Store the loss at the end of each epoch for the train              
        loss_best         = 999999999
        epoch             = 0                       
        
        # Loop for each epoch until the maximum number of epochs
        while (epoch < epochs):
            hedging_err_train = []  # Store hedging errors obtained for one complete epoch 
            np.random.shuffle(idx)  # Randomize the dataset (not useful in this case since dataset is simulated iid)
            
            # loop over each batch size
            for i in range(int(sample_size/batch_size)):
                
                # Indexes of paths used for the mini-batch
                indices = idx[i*batch_size : (i+1)*batch_size]
                                
                # SGD step 
                # - if trading stock
                if(self.hedging_instruments == "Stock"):
                    _, hedging_err = sess.run([self.train, self.hedging_err], 
                                               {self.input        : paths[:,indices,:],
                                                self.disc_tensor  : disc_batch})
                # if trading options
                elif(self.hedging_instruments == "ATM call and put"):
                    _, hedging_err = sess.run([self.train, self.hedging_err], 
                                               {self.input        : paths[:,indices,:],
                                                self.opts_price   : option_prices[:,indices,:],
                                                self.disc_tensor  : disc_batch})               
                
                hedging_err_train.append(hedging_err)
            
            # Store the loss on the train set after each epoch for learning curve
            self.loss_epochs[epoch] = self.loss_out_optim(np.concatenate(hedging_err_train))
            if(self.loss_epochs[epoch] < loss_best):
                loss_best = self.loss_epochs[epoch]
                self.saver.save(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % self.model_name)
            
            # Print the CVaR value at the end of each epoch
            print('Time elapsed:', dt.datetime.now()-start)
            print('Epoch %d, CVaR - Train: %.3f' % (epoch+1, self.loss_epochs[epoch]))
                
            epoch+=1  # increment the epoch
                
        # End of training
        print("---Finished training results---")
        print('Time elapsed:', dt.datetime.now()-start)    

        # Return the learning curve 
        return self.loss_epochs
    
    # ---------------------------------------------------------------------------------------#
    # Compute the payoff of the liability to hedge               
    # - Stricly used for the case where freq obs|rebal == monthly, but payoff freq == yearly #
    # ---------------------------------------------------------------------------------------#
    def payoff_liquid_inst_func(self):
        # Two ATM options at each time-step
        payoff_call = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[1:,:] - self.underlying_unorm_prices[0:-1,:], 0), axis=2)
        payoff_put = tf.expand_dims(tf.maximum(self.underlying_unorm_prices[0:-1,:] - self.underlying_unorm_prices[1:,:],0), axis=2)
            
        # Compilation
        payoff = tf.concat([payoff_call, payoff_put], axis=2)    
        return(payoff)
    
    def loss_out_optim(self, hedging_err):
        if (self.loss_type == "CVaR"):
            loss = np.mean(np.sort(hedging_err)[int(self.alpha*hedging_err.shape[0]):])
        return loss
    
    # option_prices: will only be used if we hedge with options
    def training(self, paths, disc_paths, sess, epochs, 
                 option_prices=-999999999999, init=True):
        
        if init:
            # for the training part, you reset all of your parameters
            sess.run(tf.global_variables_initializer()) 
        
        loss_epoch = self.train_deephedging(paths, disc_paths, sess, epochs, option_prices)

        return loss_epoch
        
    def predict(self, paths, disc_paths, sess, loss_type, option_prices=-99999999):
        sample_size = paths.shape[1]
        batch_size=self.batch_size    
        idx = np.arange(sample_size)  # [0,1,...,sample_size-1]
        start = dt.datetime.now()     # compute time
        
        # Save the hedging Pnl for each batch      
        hedging_err_pred = [] 
        strategy_pred = []
            
        # loop over sample size to do one complete epoch
        # WATCH OUT: will use the whole dataset only if sample_size/batch_size = integer...
        for i in range(int(sample_size/batch_size)):
                
            # vector of the paths we are looking at
            indices = idx[i*batch_size : (i+1)*batch_size]
            batch_disc = disc_paths[:,indices,:]          
            
            # SGD step 
            # - if trading stock
            if(self.hedging_instruments == "Stock"):
                hedging_err, strategy = sess.run([self.hedging_err, self.strategy], 
                                                 {self.input        : paths[:,indices,:],
                                                  self.disc_tensor  : batch_disc}) 
            
            # if trading two options
            elif(self.hedging_instruments == "ATM call and put"):
                
                hedging_err, strategy = sess.run([self.hedging_err, self.strategy], 
                                                 {self.input        : paths[:,indices,:],
                                                  self.opts_price   : option_prices[:,indices,:],
                                                  self.disc_tensor  : batch_disc}) 
            
            # This is the batch of input: (nbs_point_traj x indices x nbs_assets)
            strategy_pred.append(strategy)

        return np.concatenate(strategy_pred,axis=1)   
         
    def restore(self, sess, checkpoint):
        self.saver.restore(sess, checkpoint)

### 2) Construction of the train-test sets for all cases (daily|monthly underlying + two options)
- A.1) Simulation of 500K paths under the Merton jump-diffusion model with the parameters of jump risk scenario 2: 
    - [nu, sigma, lambda_, gamma, delta] = [0.111093, 0.132288, 0.25, -0.10, 0.10]
- A.2) Log AR(1) model for implied volatility
    - [kappa, theta, sigma_IV, IV_0] = [0.15, np.log(0.15), 0.06, log(0.15)]
- B) Compute option prices at each time-step for all paths
    - two options: ATM calls and puts;
- C) Split into train-test sets with 400K and 100K.

In [4]:
def BS_d1(S, dt, r, sigma, strike):
    return (np.log(S/strike) + (r+sigma**2/2)*dt) / (sigma*np.sqrt(dt))

# Style : +1 for call, -1 for put
def BlackScholes_price(S, T, r, sigma, strike, style, t=0):
    dt = T-t
    Phi = stats.norm(loc=0, scale=1).cdf
    d1 = BS_d1(S, dt, r, sigma, strike)
    d2 = d1 - sigma*np.sqrt(dt)
    return style*S*Phi(style*d1) - style*strike*np.exp(-r*dt)*Phi(style*d2)

In [ ]:
# Parameters - MJD
# nu, sigma and lambda_ are annual basis
nu, sigma, lambda_, gamma, delta = [0.111093, 0.132288, 0.25, -0.10, 0.10]
r                     = 0.03       # risk-free rate
S_0                   = 100        # Initial stock price
strike                = 100        # Strike of put option to price
T                     = 1          # Time-to-maturity of put option
n_sims                = 500000     # Total number of paths to simulate
n_timesteps_daily     = int(T*252) # Daily
n_timesteps_month     = int(T*12)  # month
n_timesteps_quarterly = int(T*4)   # Quarterly

# Parameters - IV dynamics
kappa_IV, theta_IV, sigma_IV = [0.15, np.log(0.15), 0.06]
IV_0                   = np.exp(theta_IV)
rho                    = -0.6 # correlation term

# A) Simulate daily MJD-AR(1) dynamics
h                      = T / n_timesteps_daily  # step-size
Price_mat_daily        = np.zeros((n_timesteps_daily+1, n_sims))
Price_mat_daily[0,:]   = S_0
log_IV_mat             = np.zeros((n_timesteps_daily+1, n_sims))
log_IV_mat[0,:]        = np.log(IV_0)

# Periodic parameters rescaling
nu_per                 = nu*h
sig_per                = sigma*np.sqrt(h)
jumpInt_per            = lambda_ * h      
kappa                  = np.exp(gamma + delta*delta/2) - 1 
#a                      = nu_per - jumpInt_per*(np.exp(gamma + delta*delta/2) - 1) - 0.5*sig_per*sig_per
a                      = nu_per - jumpInt_per*kappa - 0.5*sig_per*sig_per

# 1) Simulate log-returns and IV 
for i in range(n_sims):
    
    # 1.1) Simulate all log-returns for the ith path 
    Z_0     = np.random.randn(n_timesteps_daily)
    nbJumps = np.random.poisson(jumpInt_per, size=n_timesteps_daily)
    Z       = a + gamma * nbJumps + np.sqrt(sig_per*sig_per + delta*delta * nbJumps)*Z_0
    Price_mat_daily[1:,i] = S_0*np.exp(np.cumsum(Z))
                        
    # 1.2) Simulate implied volatilities from the log-AR(1) 
    for j in range(n_timesteps_daily):
        Z_1 = np.random.randn(1)        
        if(rho !=0):
            log_IV_mat[j+1,i] = log_IV_mat[j,i] + kappa_IV*(theta_IV - log_IV_mat[j,i]) + sigma_IV*(rho*Z_0[j] + np.sqrt(1-rho**2)*Z_1)
        else:
            log_IV_mat[j+1,i] = log_IV_mat[j,i] + kappa_IV*(theta_IV - log_IV_mat[j,i]) + sigma_IV*Z_1
            
IV_mat     = np.exp(log_IV_mat)
log_IV_mat = []

# B) month time-steps
idx             = np.arange(0, 253, 21)  #[0,21,...,252]
Price_mat_month = Price_mat_daily[idx,:]
IV_mat_month    = IV_mat[idx,:]

# C) Quarterly time-steps
idx             = np.arange(0, 253, 63)
Price_mat_quart = Price_mat_daily[idx,:]
IV_mat_quart    = IV_mat[idx,:]

# D) Price month and quarterly ATM call and put
Opts_month        = np.zeros((Price_mat_month.shape[0], Price_mat_month.shape[1], 2))
Opts_month[:,:,0] = BlackScholes_price(Price_mat_month, 1/12, r, IV_mat_month, Price_mat_month, 1)
Opts_month[:,:,1] = BlackScholes_price(Price_mat_month, 1/12, r, IV_mat_month, Price_mat_month, -1)
Opts_month_train = Opts_month[:,0:400000,:]
Opts_month_test  = Opts_month[:,400000:,:]

Opts_quart        = np.zeros((Price_mat_quart.shape[0], Price_mat_quart.shape[1], 2))
Opts_quart[:,:,0] = BlackScholes_price(Price_mat_quart, 3/12, r, IV_mat_quart, Price_mat_quart, 1)
Opts_quart[:,:,1] = BlackScholes_price(Price_mat_quart, 3/12, r, IV_mat_quart, Price_mat_quart, -1)
Opts_quart_train  = Opts_quart[:,0:400000,:]
Opts_quart_test   = Opts_quart[:,400000:,:]

# E) Split into train-test sets
# E.1) Daily underlying: [S_t]
train_stock_daily_input        = np.zeros((n_timesteps_daily+1, 400000,1))
test_stock_daily_input         = np.zeros((n_timesteps_daily+1, 100000,1))
train_stock_daily_input[:,:,0] = Price_mat_daily[:,0:400000]  
test_stock_daily_input[:,:,0]  = Price_mat_daily[:,400000:]

# E.2) month underlying [S_t]
train_stock_month_input        = np.zeros((n_timesteps_month+1, 400000,1))
test_stock_month_input         = np.zeros((n_timesteps_month+1, 100000,1))
train_stock_month_input[:,:,0] = Price_mat_month[:,0:400000]  
test_stock_month_input[:,:,0]  = Price_mat_month[:,400000:]

# E.3) month ATM calls and puts [S_t, IV_t]
train_opts_month_input         = np.zeros((n_timesteps_month+1, 400000,2))
test_opts_month_input          = np.zeros((n_timesteps_month+1, 100000,2))
train_opts_month_input[:,:,0]  = Price_mat_month[:,0:400000]  
test_opts_month_input[:,:,0]   = Price_mat_month[:,400000:]
train_opts_month_input[:,:,1]  = IV_mat_month[:,0:400000]
test_opts_month_input[:,:,1]   = IV_mat_month[:,400000:]

# E.4) Quarterly ATM calls and puts [S_t, IV_t]
train_opts_quart_input         = np.zeros((n_timesteps_quarterly+1, 400000,2))
test_opts_quart_input          = np.zeros((n_timesteps_quarterly+1, 100000,2))
train_opts_quart_input[:,:,0]  = Price_mat_quart[:,0:400000]  
test_opts_quart_input[:,:,0]   = Price_mat_quart[:,400000:]
train_opts_quart_input[:,:,1]  = IV_mat_quart[:,0:400000]
test_opts_quart_input[:,:,1]   = IV_mat_quart[:,400000:]

# F) Preprocessing of stock prices {log(S_n/K)}
prepro_stock = "Log-moneyness"
if(prepro_stock == "Log-moneyness"):
    train_stock_daily_input = np.log(train_stock_daily_input/strike)
    test_stock_daily_input  = np.log(test_stock_daily_input/strike)
    
    train_stock_month_input = np.log(train_stock_month_input/strike)
    test_stock_month_input  = np.log(test_stock_month_input/strike)    
    
    train_opts_month_input[:,:,0] = np.log(train_opts_month_input[:,:,0]/strike)
    test_opts_month_input[:,:,0]  = np.log(test_opts_month_input[:,:,0]/strike)
    
    train_opts_quart_input[:,:,0] = np.log(train_opts_quart_input[:,:,0]/strike)
    test_opts_quart_input[:,:,0]  = np.log(test_opts_quart_input[:,:,0]/strike)

In [6]:
batch_size     = 1000   # batch size
epochs         = 50     # number of epochs
nbs_layers     = 2      # number of LSTM cells
nbs_units      = 24     # neurons per layer  
alpha          = 0.95   # confidence level of the CVaR risk measure
lr             = 0.01/6 # learning rate hyperparameter of Adam
loss_type      = 'CVaR' 

### 3) Tensor of discount factors
- disc_mat: (n_timesteps+1 x n_sims x 1) tensor of cumulative discount rates (discount from t to 0), i.e. $\exp(-rh*n)$ 
    - Will be split into disc_mat_train and disc_mat_test for convenience

In [7]:
# Daily tensors
disc_mat_daily              = np.zeros((n_timesteps_daily+1, 500000, 1))
disc_train_daily            = np.zeros((n_timesteps_daily+1, 400000, 1))
disc_test_daily             = np.zeros((n_timesteps_daily+1, 100000, 1))
discount_vect_daily         = np.ones(disc_mat_daily.shape[0])   
h                           = T / n_timesteps_daily
discount_vect_daily[1:]     = np.exp(-r*h)              # [1,exp(-rh), exp(-rh),....,exp(-rh)]
discount_vect_daily         = np.cumprod(discount_vect_daily) # [1,exp(-rh), exp(-r2h),....,exp(-rNh)]
disc_mat_daily              = np.reshape(np.repeat(discount_vect_daily, 500000), (n_timesteps_daily+1, 500000,1))
disc_train_daily            = disc_mat_daily[:,0:400000,:]
disc_test_daily             = disc_mat_daily[:,400000::,:]
disc_batch_daily            = disc_mat_daily[:,0:batch_size,:]  # for convenience when only batch-size is needed
disc_mat_daily              = []

# month tensors
disc_mat_month              = np.zeros((n_timesteps_month+1, 500000, 1))
disc_train_month            = np.zeros((n_timesteps_month+1, 400000, 1))
disc_test_month             = np.zeros((n_timesteps_month+1, 100000, 1))
discount_vect_month         = np.ones(disc_mat_month.shape[0])   
h                           = T / n_timesteps_month
discount_vect_month[1:]     = np.exp(-r*h)              # [1,exp(-rh), exp(-rh),....,exp(-rh)]
discount_vect_month         = np.cumprod(discount_vect_month) # [1,exp(-rh), exp(-r2h),....,exp(-rNh)]
disc_mat_month              = np.reshape(np.repeat(discount_vect_month, 500000), (n_timesteps_month+1, 500000,1))
disc_train_month            = disc_mat_month[:,0:40000,:]
disc_test_month             = disc_mat_month[:,400000::,:]
disc_batch_month            = disc_mat_month[:,0:batch_size,:]  # for convenience when only batch-size is needed
disc_mat_month              = []

# Quarterly tensors
disc_mat_quart              = np.zeros((n_timesteps_quarterly+1, 500000, 1))
disc_train_quart            = np.zeros((n_timesteps_quarterly+1, 40000, 1))
disc_test_quart             = np.zeros((n_timesteps_quarterly+1, 100000, 1))
discount_vect_quart         = np.ones(disc_mat_quart.shape[0])   
h                           = T / n_timesteps_quarterly
discount_vect_quart[1:]     = np.exp(-r*h)              # [1,exp(-rh), exp(-rh),....,exp(-rh)]
discount_vect_quart         = np.cumprod(discount_vect_quart) # [1,exp(-rh), exp(-r2h),....,exp(-rNh)]
disc_mat_quart              = np.reshape(np.repeat(discount_vect_quart, 500000), (n_timesteps_quarterly+1, 500000,1))
disc_train_quart            = disc_mat_quart[:,0:400000,:]
disc_test_quart             = disc_mat_quart[:,400000::,:]
disc_batch_quart            = disc_mat_quart[:,0:batch_size,:]  # for convenience when only batch-size is needed
disc_mat_quart              = []

### 4) Function to compute the measured risk exposure given a sample of paths and hedging decisions

In [8]:
#-------------------------------------------------------------------------------------------------#
#---------------      Functions to evaluate the measured risk exposure       ---------------------#
#-------------------------------------------------------------------------------------------------#
""" 
Input:
- deltas             : (time step x nbs of paths) 
- paths              : (time step x nbs of paths)  
- disc_paths         : (time step x nbs of paths x 1)
- alpha              : for the CVaR computation
- position_type      : {Long, Short}
- hedging_instruments: {"Stock", "ATM call and put"}
- prepro_stock       : {Log, Log-moneyness, Nothing} - what transformation was used for stock prices
- option_prices      : Prices of options used as hedging instruments (not used if trading stock)
"""
#-------------------------------------------------------------------------------------------------#
def measured_risk_exposures(deltas, paths, disc_paths, strike, position_type, hedging_instruments,
                            prepro_stock, option_prices = -999999999999):
    
    # 1) Store the number of hedging instruments
    nbs_assets = deltas.shape[2] 
    
    # 2) Store prices of the underlying unormalized
    underlying_unorm_prices = inverse_processing(paths[:,:,0], prepro_stock, strike)

    # 3) Store the unormalized hedging instrument prices
    if(hedging_instruments == "Stock"):
        unorm_hedging_inst_price_b = np.expand_dims(inverse_processing(paths[0:-1,:,0], prepro_stock, strike),axis=2)
        unorm_hedging_inst_price_e = np.expand_dims(inverse_processing(paths[1:,:,0], prepro_stock, strike),axis=2)

    elif(hedging_instruments == "ATM call and put"):
        unorm_hedging_inst_price_b = option_prices[0:-1,:,:]
        unorm_hedging_inst_price_e = payoff_liquid_inst_func(underlying_unorm_prices)

  
    # 4) Store the unormalized hedging instrument prices discounted at time "0"    
    unorm_discount_hedging_inst_price_b = unorm_hedging_inst_price_b*disc_paths[0:-1,:,:]
    unorm_discount_hedging_inst_price_e = unorm_hedging_inst_price_e*disc_paths[1:,:,:]
    inc_disc_ret                        = unorm_discount_hedging_inst_price_e - unorm_discount_hedging_inst_price_b
        
    # 5) Option payoff of vanilla ATM put
    option_payoff  = np.maximum(strike - underlying_unorm_prices[-1,:],0) 
    
    # 6) Compute the hedging error
    cumulative_factor = np.reciprocal(disc_paths[-1,:,0]) 
    disc_hedging_gain = np.sum(np.sum(deltas*inc_disc_ret, axis = 0), axis = 1) # as of T
    
    if(position_type == 'Short'):
        hedging_err = option_payoff - cumulative_factor*disc_hedging_gain
    elif(position_type == 'Long'):
        hedging_err = -option_payoff - cumulative_factor*disc_hedging_gain
    
    # 7) Empirical CVaR at level alpha
    loss = np.mean(np.sort(hedging_err)[int(alpha*hedging_err.shape[0]):])
    
    return loss


def payoff_liquid_inst_func(underlying_unorm_prices): 
    payoff_call = np.expand_dims(np.maximum(underlying_unorm_prices[1:,:] - underlying_unorm_prices[0:-1,:], 0), axis=2)
    payoff_put = np.expand_dims(np.maximum(underlying_unorm_prices[0:-1,:] - underlying_unorm_prices[1:,:], 0), axis=2)

    # Compilation
    payoff = np.concatenate([payoff_call, payoff_put], axis=2)
        
    return payoff

# De-normalize the stock price process
def inverse_processing(paths, prepro_stock, strike = -9999999):
    if(prepro_stock == "Log"):
        paths = np.exp(paths)
    elif(prepro_stock == "Log-moneyness"):
        paths = strike*np.exp(paths)
    return paths

### 5) Training of the short neural network and computation of the short measured risk exposure $$\epsilon^{(S)}(0) = \underset{\delta \in \Pi}{\min} \, \rho \left(\Phi(S_{N},Z_{N}) - B_{N}G_{N}^{\delta}\right).$$ 

### 5.A) With underlying stock - daily

In [10]:
# Normalization 
nbs_assets                = 1
hedging_instruments       = "Stock"
model_name                = 'Daily_stock_short'
position_type             = "Short"

LSTM_underlying = DeepAgent(train_stock_daily_input.shape[0], batch_size, train_stock_daily_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_daily_input, disc_batch_daily, sess, epochs)
    print('---Training end---')

model_predict = DeepAgent(test_stock_daily_input.shape[0], batch_size, test_stock_daily_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_stock_daily_input, disc_test_daily, sess, loss_type)
    CVaR_95_short_test_stock_daily = measured_risk_exposures(deltas, test_stock_daily_input, 
                            disc_test_daily, strike, position_type, hedging_instruments, prepro_stock)

---Start training---
Time elapsed: 0:01:55.164542
Epoch 1, CVaR - Train: 10.540
Time elapsed: 0:03:34.698418
Epoch 2, CVaR - Train: 9.882
Time elapsed: 0:05:13.996033
Epoch 3, CVaR - Train: 9.819
Time elapsed: 0:06:52.674893
Epoch 4, CVaR - Train: 9.746
Time elapsed: 0:08:31.468835
Epoch 5, CVaR - Train: 9.721
Time elapsed: 0:10:09.170208
Epoch 6, CVaR - Train: 9.669
Time elapsed: 0:11:44.413670
Epoch 7, CVaR - Train: 9.671
Time elapsed: 0:13:22.094851
Epoch 8, CVaR - Train: 9.591
Time elapsed: 0:14:57.252480
Epoch 9, CVaR - Train: 9.639
Time elapsed: 0:16:32.511121
Epoch 10, CVaR - Train: 9.607
Time elapsed: 0:18:10.283334
Epoch 11, CVaR - Train: 9.543
Time elapsed: 0:19:48.029540
Epoch 12, CVaR - Train: 9.472
Time elapsed: 0:21:23.088127
Epoch 13, CVaR - Train: 9.498
Time elapsed: 0:22:58.238753
Epoch 14, CVaR - Train: 9.525
Time elapsed: 0:24:36.086983
Epoch 15, CVaR - Train: 9.455
Time elapsed: 0:26:11.363618
Epoch 16, CVaR - Train: 9.457
Time elapsed: 0:27:49.198855
Epoch 17, CVaR

### 5.B) With underlying stock - month

In [12]:
# Normalization 
nbs_assets                = 1
hedging_instruments       = "Stock"
model_name                = 'Month_stock_short'
position_type             = "Short"

LSTM_underlying = DeepAgent(train_stock_month_input.shape[0], batch_size, train_stock_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_month_input, disc_batch_month, sess, epochs)
    print('---Training end---')
        
model_predict = DeepAgent(test_stock_month_input.shape[0], batch_size, test_stock_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_stock_month_input, disc_test_month, sess, loss_type)
    CVaR_95_short_test_stock_month = measured_risk_exposures(deltas, test_stock_month_input, 
                            disc_test_month, strike, position_type, hedging_instruments, prepro_stock)

---Start training---
Time elapsed: 0:00:05.962354
Epoch 1, CVaR - Train: 11.347
Time elapsed: 0:00:11.311752
Epoch 2, CVaR - Train: 10.485
Time elapsed: 0:00:16.645973
Epoch 3, CVaR - Train: 10.381
Time elapsed: 0:00:21.970424
Epoch 4, CVaR - Train: 10.300
Time elapsed: 0:00:27.294624
Epoch 5, CVaR - Train: 10.228
Time elapsed: 0:00:32.611842
Epoch 6, CVaR - Train: 10.189
Time elapsed: 0:00:37.939200
Epoch 7, CVaR - Train: 10.143
Time elapsed: 0:00:43.262583
Epoch 8, CVaR - Train: 10.112
Time elapsed: 0:00:48.575616
Epoch 9, CVaR - Train: 10.085
Time elapsed: 0:00:53.880968
Epoch 10, CVaR - Train: 10.076
Time elapsed: 0:00:59.213179
Epoch 11, CVaR - Train: 10.037
Time elapsed: 0:01:04.628481
Epoch 12, CVaR - Train: 10.017
Time elapsed: 0:01:10.015830
Epoch 13, CVaR - Train: 10.016
Time elapsed: 0:01:15.368737
Epoch 14, CVaR - Train: 10.005
Time elapsed: 0:01:20.754933
Epoch 15, CVaR - Train: 9.990
Time elapsed: 0:01:26.107174
Epoch 16, CVaR - Train: 9.981
Time elapsed: 0:01:31.477176
E

### 5.C) With two opts - month

In [14]:
# Normalization 
nbs_assets                = 2
hedging_instruments       = "ATM call and put"
model_name                = 'Month_opts_short'
position_type             = "Short"

LSTM_underlying = DeepAgent(train_opts_month_input.shape[0], batch_size, train_opts_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_opts_month_input, disc_batch_month, sess, epochs,
                                            Opts_month_train)
    print('---Training end---')
        
model_predict = DeepAgent(test_opts_month_input.shape[0], batch_size, test_opts_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_opts_month_input, disc_test_month, sess, loss_type, Opts_month_test)
    CVaR_95_short_test_opts_month = measured_risk_exposures(deltas, test_opts_month_input, 
                            disc_test_month, strike, position_type, hedging_instruments, prepro_stock, Opts_month_test)

---Start training---
Time elapsed: 0:00:07.065906
Epoch 1, CVaR - Train: 9.660
Time elapsed: 0:00:13.358501
Epoch 2, CVaR - Train: 8.207
Time elapsed: 0:00:19.642262
Epoch 3, CVaR - Train: 8.111
Time elapsed: 0:00:25.913687
Epoch 4, CVaR - Train: 8.009
Time elapsed: 0:00:32.191140
Epoch 5, CVaR - Train: 7.919
Time elapsed: 0:00:38.471867
Epoch 6, CVaR - Train: 7.877
Time elapsed: 0:00:44.743694
Epoch 7, CVaR - Train: 7.845
Time elapsed: 0:00:51.050126
Epoch 8, CVaR - Train: 7.796
Time elapsed: 0:00:57.314576
Epoch 9, CVaR - Train: 7.767
Time elapsed: 0:01:03.613133
Epoch 10, CVaR - Train: 7.764
Time elapsed: 0:01:09.906981
Epoch 11, CVaR - Train: 7.752
Time elapsed: 0:01:16.192742
Epoch 12, CVaR - Train: 7.726
Time elapsed: 0:01:22.470566
Epoch 13, CVaR - Train: 7.699
Time elapsed: 0:01:28.772493
Epoch 14, CVaR - Train: 7.679
Time elapsed: 0:01:34.899924
Epoch 15, CVaR - Train: 7.679
Time elapsed: 0:01:41.203749
Epoch 16, CVaR - Train: 7.645
Time elapsed: 0:01:47.464300
Epoch 17, CVaR 

### 5.D) With two opts - quarterly

In [15]:
# Normalization 
nbs_assets                = 2
hedging_instruments       = "ATM call and put"
model_name                = 'Quart_opts_short'
position_type             = "Short"

LSTM_underlying = DeepAgent(train_opts_quart_input.shape[0], batch_size, train_opts_quart_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_opts_quart_input, disc_batch_quart, sess, epochs,
                                            Opts_quart_train)
    print('---Training end---')
        
model_predict = DeepAgent(test_opts_quart_input.shape[0], batch_size, test_opts_quart_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_opts_quart_input, disc_test_quart, sess, loss_type, Opts_quart_test)
    CVaR_95_short_test_opts_quart = measured_risk_exposures(deltas, test_opts_quart_input, 
                            disc_test_quart, strike, position_type, hedging_instruments, prepro_stock, Opts_quart_test)

---Start training---
Time elapsed: 0:00:02.926702
Epoch 1, CVaR - Train: 9.749
Time elapsed: 0:00:05.569303
Epoch 2, CVaR - Train: 7.779
Time elapsed: 0:00:08.220551
Epoch 3, CVaR - Train: 7.619
Time elapsed: 0:00:10.879024
Epoch 4, CVaR - Train: 7.550
Time elapsed: 0:00:13.523908
Epoch 5, CVaR - Train: 7.520
Time elapsed: 0:00:16.175710
Epoch 6, CVaR - Train: 7.479
Time elapsed: 0:00:18.944340
Epoch 7, CVaR - Train: 7.475
Time elapsed: 0:00:21.600944
Epoch 8, CVaR - Train: 7.447
Time elapsed: 0:00:24.252685
Epoch 9, CVaR - Train: 7.413
Time elapsed: 0:00:26.900368
Epoch 10, CVaR - Train: 7.398
Time elapsed: 0:00:29.548428
Epoch 11, CVaR - Train: 7.365
Time elapsed: 0:00:32.196367
Epoch 12, CVaR - Train: 7.356
Time elapsed: 0:00:34.760376
Epoch 13, CVaR - Train: 7.368
Time elapsed: 0:00:37.418122
Epoch 14, CVaR - Train: 7.350
Time elapsed: 0:00:40.066465
Epoch 15, CVaR - Train: 7.347
Time elapsed: 0:00:42.703064
Epoch 16, CVaR - Train: 7.338
Time elapsed: 0:00:45.340781
Epoch 17, CVaR 

### 6) Training of the long neural network and computation of the long measured risk exposure 
$$\epsilon^{(L)}(0) = \underset{\delta\in \Pi}{\min} \, \rho \left(-\Phi(S_{N},Z_{N}) -B_{N}G_{N}^{\delta}\right).$$

### 6.A) With underlying stock - daily

In [16]:
# Normalization 
nbs_assets                = 1
hedging_instruments       = "Stock"
model_name                = 'Daily_stock_long'
position_type             = "Long"

LSTM_underlying = DeepAgent(train_stock_daily_input.shape[0], batch_size, train_stock_daily_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_daily_input, disc_batch_daily, sess, epochs)
    print('---Training end---')
        
model_predict = DeepAgent(test_stock_daily_input.shape[0], batch_size, test_stock_daily_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_stock_daily_input, disc_test_daily, sess, loss_type)
    CVaR_95_long_test_stock_daily = measured_risk_exposures(deltas, test_stock_daily_input, 
                            disc_test_daily, strike, position_type, hedging_instruments, prepro_stock)

---Start training---
Time elapsed: 0:01:54.108914
Epoch 1, CVaR - Train: -0.362
Time elapsed: 0:03:32.033170
Epoch 2, CVaR - Train: -2.791
Time elapsed: 0:05:09.872653
Epoch 3, CVaR - Train: -2.986
Time elapsed: 0:06:47.689865
Epoch 4, CVaR - Train: -3.043
Time elapsed: 0:08:25.694130
Epoch 5, CVaR - Train: -3.054
Time elapsed: 0:10:01.041802
Epoch 6, CVaR - Train: -3.040
Time elapsed: 0:11:36.438476
Epoch 7, CVaR - Train: -2.956
Time elapsed: 0:13:14.287705
Epoch 8, CVaR - Train: -3.113
Time elapsed: 0:14:52.383514
Epoch 9, CVaR - Train: -3.164
Time elapsed: 0:16:30.112717
Epoch 10, CVaR - Train: -3.196
Time elapsed: 0:18:07.928938
Epoch 11, CVaR - Train: -3.240
Time elapsed: 0:19:45.775160
Epoch 12, CVaR - Train: -3.271
Time elapsed: 0:21:23.699415
Epoch 13, CVaR - Train: -3.313
Time elapsed: 0:23:01.565648
Epoch 14, CVaR - Train: -3.320
Time elapsed: 0:24:39.553901
Epoch 15, CVaR - Train: -3.337
Time elapsed: 0:26:17.281281
Epoch 16, CVaR - Train: -3.345
Time elapsed: 0:27:55.016485

### 6.B) With underlying stock - month

In [17]:
# Normalization 
nbs_assets                = 1
hedging_instruments       = "Stock"
model_name                = 'Month_stock_long'
position_type             = "Long"

LSTM_underlying = DeepAgent(train_stock_month_input.shape[0], batch_size, train_stock_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_stock_month_input, disc_batch_month, sess, epochs)
    print('---Training end---')
        
model_predict = DeepAgent(test_stock_month_input.shape[0], batch_size, test_stock_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_stock_month_input, disc_test_month, sess, loss_type)
    CVaR_95_long_test_stock_month = measured_risk_exposures(deltas, test_stock_month_input, 
                            disc_test_month, strike, position_type, hedging_instruments, prepro_stock)

---Start training---
Time elapsed: 0:00:05.999924
Epoch 1, CVaR - Train: -0.454
Time elapsed: 0:00:11.410364
Epoch 2, CVaR - Train: -1.816
Time elapsed: 0:00:16.797264
Epoch 3, CVaR - Train: -1.908
Time elapsed: 0:00:22.210824
Epoch 4, CVaR - Train: -1.947
Time elapsed: 0:00:27.614381
Epoch 5, CVaR - Train: -1.963
Time elapsed: 0:00:33.024898
Epoch 6, CVaR - Train: -1.979
Time elapsed: 0:00:38.424125
Epoch 7, CVaR - Train: -1.997
Time elapsed: 0:00:43.834354
Epoch 8, CVaR - Train: -1.999
Time elapsed: 0:00:49.244783
Epoch 9, CVaR - Train: -2.002
Time elapsed: 0:00:54.646999
Epoch 10, CVaR - Train: -2.007
Time elapsed: 0:01:00.042236
Epoch 11, CVaR - Train: -2.019
Time elapsed: 0:01:05.318993
Epoch 12, CVaR - Train: -2.018
Time elapsed: 0:01:10.734472
Epoch 13, CVaR - Train: -2.021
Time elapsed: 0:01:16.135829
Epoch 14, CVaR - Train: -2.023
Time elapsed: 0:01:21.528404
Epoch 15, CVaR - Train: -2.027
Time elapsed: 0:01:26.926795
Epoch 16, CVaR - Train: -2.028
Time elapsed: 0:01:32.326022

### 6.C) With two opts - month

In [18]:
# Normalization 
nbs_assets                = 2
hedging_instruments       = "ATM call and put"
model_name                = 'Month_opts_long'
position_type             = "Long"

LSTM_underlying = DeepAgent(train_opts_month_input.shape[0], batch_size, train_opts_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_opts_month_input, disc_batch_month, sess, epochs,
                                            Opts_month_train)
    print('---Training end---')
        
model_predict = DeepAgent(test_opts_month_input.shape[0], batch_size, test_opts_month_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_opts_month_input, disc_test_month, sess, loss_type, Opts_month_test)
    CVaR_95_long_test_opts_month = measured_risk_exposures(deltas, test_opts_month_input, 
                            disc_test_month, strike, position_type, hedging_instruments, prepro_stock, Opts_month_test)

---Start training---
Time elapsed: 0:00:07.066877
Epoch 1, CVaR - Train: -0.477
Time elapsed: 0:00:13.537624
Epoch 2, CVaR - Train: -3.396
Time elapsed: 0:00:20.008914
Epoch 3, CVaR - Train: -3.634
Time elapsed: 0:00:26.470753
Epoch 4, CVaR - Train: -3.756
Time elapsed: 0:00:32.959620
Epoch 5, CVaR - Train: -3.802
Time elapsed: 0:00:39.428027
Epoch 6, CVaR - Train: -3.822
Time elapsed: 0:00:45.906805
Epoch 7, CVaR - Train: -3.830
Time elapsed: 0:00:52.369986
Epoch 8, CVaR - Train: -3.840
Time elapsed: 0:00:58.841594
Epoch 9, CVaR - Train: -3.856
Time elapsed: 0:01:05.313483
Epoch 10, CVaR - Train: -3.860
Time elapsed: 0:01:11.598037
Epoch 11, CVaR - Train: -3.852
Time elapsed: 0:01:18.079499
Epoch 12, CVaR - Train: -3.869
Time elapsed: 0:01:24.383941
Epoch 13, CVaR - Train: -3.865
Time elapsed: 0:01:30.851401
Epoch 14, CVaR - Train: -3.875
Time elapsed: 0:01:37.313868
Epoch 15, CVaR - Train: -3.884
Time elapsed: 0:01:43.615300
Epoch 16, CVaR - Train: -3.876
Time elapsed: 0:01:50.078779

### 6.D) With two opts - quarterly

In [19]:
# Normalization 
nbs_assets                = 2
hedging_instruments       = "ATM call and put"
model_name                = 'Quart_opts_short'
position_type             = "Long"

LSTM_underlying = DeepAgent(train_opts_quart_input.shape[0], batch_size, train_opts_quart_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

# Start training 
start = dt.datetime.now()

print('---Start training---')
with tf.Session() as sess:           
    loss_epoch = LSTM_underlying.training(train_opts_quart_input, disc_batch_quart, sess, epochs,
                                            Opts_quart_train)
    print('---Training end---')
        
model_predict = DeepAgent(test_opts_quart_input.shape[0], batch_size, test_opts_quart_input.shape[2], 
        loss_type, nbs_assets, position_type, nbs_layers, nbs_units, lr, prepro_stock, 
        hedging_instruments, alpha, name = model_name)

with tf.Session() as sess:
    
    # Load the saved model
    model_predict.restore(sess, r"/Users/alexa/Github files/ERP with multi hedging instruments/Models/%s/models.ckpt" % model_name)
    deltas = model_predict.predict(test_opts_quart_input, disc_test_quart, sess, loss_type, Opts_quart_test)
    CVaR_95_long_test_opts_quart = measured_risk_exposures(deltas, test_opts_quart_input, 
                            disc_test_quart, strike, position_type, hedging_instruments, prepro_stock, Opts_quart_test)

---Start training---
Time elapsed: 0:00:02.899668
Epoch 1, CVaR - Train: -0.159
Time elapsed: 0:00:05.567522
Epoch 2, CVaR - Train: -3.003
Time elapsed: 0:00:08.224564
Epoch 3, CVaR - Train: -3.215
Time elapsed: 0:00:10.881459
Epoch 4, CVaR - Train: -3.289
Time elapsed: 0:00:13.532667
Epoch 5, CVaR - Train: -3.327
Time elapsed: 0:00:16.175504
Epoch 6, CVaR - Train: -3.339
Time elapsed: 0:00:18.841496
Epoch 7, CVaR - Train: -3.342
Time elapsed: 0:00:21.489474
Epoch 8, CVaR - Train: -3.352
Time elapsed: 0:00:24.144387
Epoch 9, CVaR - Train: -3.358
Time elapsed: 0:00:26.709458
Epoch 10, CVaR - Train: -3.355
Time elapsed: 0:00:29.269039
Epoch 11, CVaR - Train: -3.358
Time elapsed: 0:00:31.916814
Epoch 12, CVaR - Train: -3.361
Time elapsed: 0:00:34.568692
Epoch 13, CVaR - Train: -3.367
Time elapsed: 0:00:37.227510
Epoch 14, CVaR - Train: -3.371
Time elapsed: 0:00:39.886132
Epoch 15, CVaR - Train: -3.375
Time elapsed: 0:00:42.449046
Epoch 16, CVaR - Train: -3.375
Time elapsed: 0:00:45.014803

### 7) Compute equal risk option prices $C_{0}^{\star}$ and incompleteness measure $\epsilon^{\star}$ with the results of the test set for all four cases
$$C_{0}^{\star} = \frac{\epsilon^{(S)}(0) - \epsilon^{(L)}(0)}{2B_{N}}, \quad \epsilon^{\star} = \frac{\epsilon^{(L)}(0) + \epsilon^{(S)}(0)}{2}.$$

In [20]:
# A) Daily stock
ERP_stock_daily     = (CVaR_95_short_test_stock_daily - CVaR_95_long_test_stock_daily)/(2*np.exp(r*T))
epsilon_stock_daily = (CVaR_95_long_test_stock_daily + CVaR_95_short_test_stock_daily)/2
print("----------------------------------------------------")
print("Daily stock hedging results:")
print("Equal risk price of ATM put of 60 days: %.4f" %(ERP_stock_daily))
print("Residual hedging risk of ATM put of 60 days: %.4f" %(epsilon_stock_daily))

# B) month stock
ERP_stock_month     = (CVaR_95_short_test_stock_month - CVaR_95_long_test_stock_month)/(2*np.exp(r*T))
epsilon_stock_month = (CVaR_95_long_test_stock_month + CVaR_95_short_test_stock_month)/2
print("----------------------------------------------------")
print("Monthly stock hedging results:")
print("Equal risk price of ATM put of 60 days: %.4f" %(ERP_stock_month))
print("Residual hedging risk of ATM put of 60 days: %.4f" %(epsilon_stock_month))

# C) month opts
ERP_opts_month     = (CVaR_95_short_test_opts_month - CVaR_95_long_test_opts_month)/(2*np.exp(r*T))
epsilon_opts_month = (CVaR_95_long_test_opts_month + CVaR_95_short_test_opts_month)/2
print("----------------------------------------------------")
print("1-month opts hedging results:")
print("Equal risk price of ATM put of 60 days: %.4f" %(ERP_opts_month))
print("Residual hedging risk of ATM put of 60 days: %.4f" %(epsilon_opts_month))

# D) Quarterly opts
ERP_opts_quarterly     = (CVaR_95_short_test_opts_quart - CVaR_95_long_test_opts_quart)/(2*np.exp(r*T))
epsilon_opts_quarterly = (CVaR_95_long_test_opts_quart + CVaR_95_short_test_opts_quart)/2
print("----------------------------------------------------")
print("3-months opts hedging results:")
print("Equal risk price of ATM put of 60 days: %.4f" %(ERP_opts_quarterly))
print("Residual hedging risk of ATM put of 60 days: %.4f" %(epsilon_opts_quarterly))

----------------------------------------------------
Daily stock hedging results:
Equal risk price of ATM put of 60 days: 6.0416
Residual hedging risk of ATM put of 60 days: 2.7642
----------------------------------------------------
Monthly stock hedging results:
Equal risk price of ATM put of 60 days: 5.7291
Residual hedging risk of ATM put of 60 days: 3.8417
----------------------------------------------------
1-month opts hedging results:
Equal risk price of ATM put of 60 days: 5.5215
Residual hedging risk of ATM put of 60 days: 1.7542
----------------------------------------------------
3-months opts hedging results:
Equal risk price of ATM put of 60 days: 5.1450
Residual hedging risk of ATM put of 60 days: 1.8853


References
- Buehler, H., Gonon, L., Teichmann, J., and Wood, B. (2019). Deep hedging. Quantitative Finance, 19(8):1271-1291.
- Carbonneau, A. and Godin, F. (2020). Equal risk pricing of derivatives with deep hedging. Quantitative Finance, pages 1-16.
- Carbonneau, A. and Godin, F. (2021). Deep Equal Risk Pricing of Financial Derivatives with Multiple Hedging Instruments. arXiv preprint arXiv:2102.12694.